In [1]:
import pandas as pd
import unicodedata


In [2]:
def remove_non_ascii_word(word):
    """Remove non-ASCII characters from list of tokenized words"""
    new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    return new_word
def hasAuthor(string):
    return autoresID.loc[autoresID['nome'] == string]

In [111]:
import pickle
with open('h2.pickle', 'rb') as f:
    W = pickle.load(f)
W = W.transpose()

In [4]:
df_artigos = pd.read_csv('../../datasets/artigos/preprocessed/without_spacy/artigos2017_pre',sep="|",index_col=0)

In [5]:
autoresID = pd.read_csv('../../datasets/autores/autores.csv', delimiter="|")
autoresID.columns=['id','nome','path']
print(len(autoresID['nome'].unique()))
autoresID.shape
autoresID['nome'] = autoresID['nome'].apply(remove_non_ascii_word)
autoresID['nome'] = autoresID['nome'].apply(lambda x: x.lower().rstrip().lstrip())
autoresCIT = pd.read_csv('../../datasets/autores/autor_cit.csv',delimiter="|")
autoresCIT.columns=['id','nome_cit']
autoresCIT['nome_cit'] = autoresCIT['nome_cit'].apply(lambda x: x.rstrip().lstrip())

7411


In [112]:
autor_topic = {}
autor_qt = {}
miss_author = 0
for idx_doc,doc in enumerate(W):
    autores = df_artigos.iloc[idx_doc]['autores']
    tempA = []
    for autor in autores.split("&"):
        autor = remove_non_ascii_word(autor.lower())
        dfAuthorName = hasAuthor(autor)
        if len(dfAuthorName) != 0:
            autorCod = dfAuthorName.iloc[0]['id'].item()
            if autor not in tempA:
                tempA.append(autorCod)
                if autorCod not in autor_topic.keys():
                    autor_topic[autorCod] = doc
                    autor_qt[autorCod] = 1
                else:
                    autor_topic[autorCod] += doc
                    autor_qt[autorCod] += 1


        else:
            df_autor_id = autoresCIT.loc[autoresCIT['nome_cit'] == autor]['id']
            if len(df_autor_id) != 0:
                autor_id = df_autor_id.item()
                autor = autoresID.loc[autoresID['id']==autor_id]['nome'].item()
                autor = remove_non_ascii_word(autor.lower())
                if autor not in tempA:
                    tempA.append(autor_id)
                    if autor_id not in autor_topic.keys():
                        autor_topic[autor_id] = doc
                        autor_qt[autor_id] = 1
                    else:
                        autor_topic[autor_id] += doc
                        autor_qt[autor_id] += 1
            #else:
                #if autor not in tempA:
                 #   tempA.append(autor)
                  #  if autor not in autor_topic.keys():
                   #     autor_topic[autor] = doc
                    #    autor_qt[autor] = 1
                    #else:
                     #   autor_topic[autor] += doc + 1
                      #  autor_qt[autor] += 1

In [113]:
autor_topic_relevance_matrix = [autor_topic[key] for key in autor_topic.keys()]

In [114]:
a_topic_relevance_matrix_normalized = []
for vec in autor_topic_relevance_matrix:
    vec = vec/max(vec)
    a_topic_relevance_matrix_normalized.append(vec)

/home/leandror/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [115]:
most_relevant_author_in_topic_matrix_normalized = []
autor_topic_relevance_matrix = np.array(autor_topic_relevance_matrix)
for vec in autor_topic_relevance_matrix.transpose():
    vec1 = vec/max(vec)
    most_relevant_author_in_topic_matrix_normalized.append(vec1)

In [116]:
table_topic_autor = []
for id_top in range(len(most_relevant_author_in_topic_matrix_normalized)):
    for i in np.argsort(-np.array(most_relevant_author_in_topic_matrix_normalized)[id_top]):
        id_autor, rank = list(autor_topic.keys())[i],most_relevant_author_in_topic_matrix_normalized[id_top][i]
        rank = rank * 100
        table_topic_autor.append([id_autor,rank,'2017','2017'+str(id_autor)+str(id_top),id_top+1])

In [117]:
pd.DataFrame(table_topic_autor).to_csv('topic_autor_h2',index=False)

In [118]:
table_autor_topic = []
for idx,i in enumerate(a_topic_relevance_matrix_normalized):
    autor_id = list(autor_topic.keys())[idx]
    for id_topic,j in enumerate(i):
        if np.isnan(j):
            j = 0
        j = j*100
        table_autor_topic.append([autor_id,j,id_topic+1,'2017',str(autor_id)+str(id_topic+1)+'2017'])

In [119]:
pd.DataFrame(table_autor_topic).to_csv('autor_topic_h2',index=False)